# Chars74K X EMNIST

Para treinar um modelo que receba uma **imagem qualquer** e diga se há letras ou números, o **Chars74K** é o **melhor dataset**.
A escolha se baseia na natureza dos dados e no objetivo do seu modelo de funcionar em um ambiente "qualquer" ou "arbitrário".

## Por que o Chars74K é superior (Recomendado)

O dataset **Chars74K** (Character Recognition in Natural Images) foi projetado para simular o mundo real. Ele possui três domínios de imagens, sendo o mais crucial o primeiro:

1.  **Caracteres Naturais:** Imagens de caracteres capturadas em ambientes reais (placas de rua, embalagens, outdoors, livros, etc.).
2.  **Caracteres Sintetizados:** Imagens de caracteres geradas por computador com diferentes fontes.
3.  **Caracteres Manuscritos:** Caracteres escritos à mão.

Ao incluir os **Caracteres Naturais**, o Chars74K força sua CNN a aprender a identificar letras e números mesmo quando eles estão:
* Em fundos complexos e desordenados.
* Com variações de iluminação e cor.
* Em diferentes fontes e tamanhos.

Isso torna o modelo muito mais **robusto** e capaz de generalizar para a tarefa de "detecção de presença" em qualquer imagem que você fornecer.

***

## Por que o EMNIST é menos adequado

O **EMNIST** (Extended MNIST) é excelente para a **classificação** de caracteres, mas é limitado para a detecção em imagens arbitrárias.

* **Conteúdo:** Imagens de caracteres manuscritos (dígitos e letras), isolados, centralizados e em fundos limpos (geralmente pretos e brancos ou em escala de cinza).
* **Limitação:** Um modelo treinado no EMNIST terá um desempenho muito bom na classificação de caracteres isolados, mas terá **dificuldade em identificar** um caractere que esteja embutido em um **fundo real** (por exemplo, o número de uma placa de carro ou o texto de uma embalagem de produto). Ele não foi treinado para lidar com o "ruído" e a complexidade visual do mundo real.

**Em resumo:** Use **Chars74K** para que seu modelo aprenda a ver caracteres em um **contexto** real. Use **EMNIST** apenas se você souber que as imagens de entrada serão sempre caracteres limpos e isolados.

---

## Formato da Imagem: PNG vs. JPG

Se todas as suas imagens já estão em **PNG**, o melhor é **continuar usando o PNG**.

| Formato | Vantagens | Desvantagens | Recomendação para CNN |
| :--- | :--- | :--- | :--- |
| **PNG** | **Sem Perda (Lossless):** Preserva a qualidade original, crucial para caracteres. Suporta transparência (não relevante aqui). | Tamanhos de arquivo **maiores** (consome mais espaço em disco e RAM). | **Melhor Escolha:** Como o Chars74K inclui caracteres com bordas limpas e detalhadas, a **qualidade sem perda** do PNG é superior para evitar que a compressão do JPG introduza ruído ou distorça pequenos detalhes. |
| **JPG** | **Com Perda (Lossy):** Tamanhos de arquivo **menores** (mais rápido de carregar e menos espaço). | A compressão pode introduzir artefatos (ruído) que a CNN pode aprender incorretamente. | Bom para fotos genéricas, mas **evite** para dados de caracteres onde a precisão de borda é importante. |

**Conclusão:** Mantenha o formato **PNG**. A perda de qualidade do JPG é um risco desnecessário para um problema de classificação de caracteres.

---

## Pré-processamento e Redimensionamento

O pré-processamento, especialmente o **redimensionamento**, é melhor ser feito **diretamente no *script* de treinamento**, usando as funcionalidades do *framework* (TensorFlow/Keras ou PyTorch).

### Por que Fazer no Script de Treinamento?

Fazer o pré-processamento na hora do treinamento (*on-the-fly*) traz várias vantagens, especialmente a flexibilidade para o *Data Augmentation*:

* **1. Facilidade de *Data Augmentation*:** Se você fizer o redimensionamento e a normalização dentro do *pipeline* de dados (e.g., usando `ImageDataGenerator` ou `tf.data`), você pode aplicar técnicas de **aumento de dados (*data augmentation*)** (rotação, zoom, etc.) **depois** do redimensionamento, mas **antes** de alimentar a CNN. Isso é crucial para a robustez do seu modelo no Chars74K.
* **2. Consistência:** Garante que cada imagem passará exatamente pela mesma sequência de transformações (redimensionamento $\to$ normalização $\to$ *augmentation*) a cada época.
* **3. Economia de Armazenamento:** Você não precisa criar uma cópia física de todo o dataset redimensionado em seu disco, economizando muito espaço. O *pipeline* lê o arquivo original e o transforma na memória.

### O que Incluir no Pré-processamento (Dentro do Script)

1.  **Redimensionamento (*Resizing*):** Todas as imagens devem ter o mesmo tamanho antes de serem alimentadas na CNN (ex: $64 \times 64$ pixels).
2.  **Normalização:** Dividir os valores dos pixels por 255 para colocá-los no intervalo $[0, 1]$.
3.  ***Data Augmentation:*** Adicionar transformações aleatórias (rotação, deslocamento, zoom) **apenas** ao conjunto de **treinamento**.